In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import keras
import json
import pickle
from keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input, decode_predictions
from keras.preprocessing import image
from keras.models import Model, load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from keras.layers import Input, Dense, Dropout, Embedding, LSTM
from keras.layers import add

In [10]:
model = load_model("model_9.h5")

In [11]:
model_temp = ResNet50(weights="imagenet",input_shape=(224,224,3))

In [12]:
#Create a new model, by removing the last layer (output layer of 1000 classes) from resnet50
model_resnet = Model(model_temp.input,model_temp.layers[-2].output)

In [13]:
import tensorflow as tf
def preprocess_img(img):
    img = tf.keras.utils.load_img(img,target_size=(224,224))
    img = tf.keras.utils.img_to_array(img)
    img = np.expand_dims(img,axis=0)
    # Normalisation
    img = preprocess_input(img)
    return img

In [14]:
def encode_image(img):
    img = preprocess_img(img)
    feature_vector = model_resnet.predict(img)
    
    feature_vector = feature_vector.reshape(1, feature_vector.shape[1])
    #print(feature_vector.shape)
    return feature_vector

In [18]:
enc = encode_image("dog.jpg")

1/1 [==============================] - 0s 119ms/step


In [19]:
enc.shape

(1, 2048)

In [20]:
with open("C:/Users/yaksh/Downloads/project3/storage/word_to_idx.pkl", "rb") as w2i:
    word_to_idx = pickle.load(w2i)
    
with open("C:/Users/yaksh/Downloads/project3/storage/idx_to_word.pkl", "rb") as i2w:
    idx_to_word = pickle.load(i2w)

In [21]:
def predict_caption(photo):
    max_len=35
    in_text = "startseq"
    for i in range(max_len):
        sequence = [word_to_idx[w] for w in in_text.split() if w in word_to_idx]
        sequence = pad_sequences([sequence],maxlen=max_len,padding='post')
        
        ypred = model.predict([photo,sequence])
        ypred = ypred.argmax() #WOrd with max prob always - Greedy Sampling
        word = idx_to_word[ypred]
        in_text += (' ' + word)
        
        if word == "endseq":
            break
    
    final_caption = in_text.split()[1:-1]
    final_caption = ' '.join(final_caption)
    return final_caption

In [22]:
predict_caption(enc)

1/1 [==============================] - 0s 31ms/step


'dog running in the sand'

In [23]:
caption=predict_caption(enc)

1/1 [==============================] - 0s 31ms/step


In [24]:
from gtts import gTTS
import os

In [25]:
tts = gTTS(text=caption,lang='en')
tts.save("C://Users//yaksh//Downloads//project3//test.mp3")

In [26]:
import IPython
IPython.display.Audio("test.mp3")